In [1]:
import numpy as np
import cv2 as cv
import glob

In [7]:
def project_point_to_image(pt_3d, rvec, tvec, camera_matrix):
    '''project a 3D point to 2D ausing camera parameters'''

    # convert rotation vector to rotation matrix
    rotation_vector, _ = cv.Rodrigues(rvec)

    # create the [R | t] matrix by concatenationg Rrotation matroc and translation vector
    rt_matrix = np.hstack((rotation_vector,tvec))

    # calculate projection matrix by multipying camera matrix with [R | t]
    projection_matrix = camera_matrix @ rt_matrix

    # convert 3D point to homogeneous transformation
    homogeneous_point = np.array([pt_3d[0], pt_3d[1], pt_3d[2], 1]).reshape(4,1)

    # project 3d point to 2d image plane
    image_point_homogeneous = projection_matrix @ homogeneous_point

    # normalize to convert from homogeneous to 2D
    image_point_homogeneous /= image_point_homogeneous[2]

    return(int(image_point_homogeneous[0][0]), int(image_point_homogeneous[1][0]))

In [4]:
# (horizontal_cubes-1, vertical_cubes-1)
CHECKERBOARD = (10,7)

# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('IMG_*.JPG')

for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    gray = cv.GaussianBlur(gray, (5, 5), 0)  # reduce noise

    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners2)

        for corner in corners2:
            #cv.circle(img, tuple(corner.ravel()), 5, (0, 255, 0), -1)  # radius 5, green color
            cv.circle(img, (int(corner.ravel()[0]), int(corner.ravel()[1])), 20, (0, 255, 0), -1) # circle radius

        for i in range(len(corners2) - 1):
            # Draw a line between consecutive corners
            pt1 = (int(corners2[i].ravel()[0]), int(corners2[i].ravel()[1]))
            pt2 = (int(corners2[i + 1].ravel()[0]), int(corners2[i + 1].ravel()[1]))  
            cv.line(img, pt1, pt2, color=(0, 255, 0), thickness=3)  # line thickness

        # Draw and display the corners
        cv.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        #cv.imshow('img', img)
        cv.imwrite(f'corner_plotted_{fname}', img)
        print(f"Saved corner_plotted_{fname}")
        #cv.waitKey(500)
    else:
        print(f"{fname}: failed to identify corner")
#cv.destroyAllWindows()

Saved corner_plotted_IMG_1052.JPG
Saved corner_plotted_IMG_1053.JPG
Saved corner_plotted_IMG_1056.JPG
Saved corner_plotted_IMG_1049.JPG
Saved corner_plotted_IMG_1047.JPG
Saved corner_plotted_IMG_1048.JPG
Saved corner_plotted_IMG_1050.JPG


In [5]:
#calibration
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Calibration successful:", ret)
print("\n\nCamera Matrix (Intrinsic Parameter):\n", mtx)
print("\n\nDistortion Coefficients:\n", dist)
print("\n\nRotation Vectors:")
print(rvecs)
print("\n\nTranslation Vectors:")
print(tvecs)

Calibration successful: 1.916697283295477


Camera matrix (Intrinsic Parameter):
 [[4.09981646e+03 0.00000000e+00 2.16941895e+03]
 [0.00000000e+00 4.09929477e+03 2.87915461e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


Distortion coefficients:
 [[ 2.16611005e-01 -9.76743333e-01  9.19770530e-04  1.56579315e-03
   1.42883584e+00]]


rotatiion vectors:
(array([[-0.53781944],
       [-0.20421435],
       [-0.920612  ]]), array([[-0.58223549],
       [-0.0014828 ],
       [ 0.00273837]]), array([[-0.00966828],
       [-0.00472928],
       [ 1.57929622]]), array([[-0.09111957],
       [-0.02846379],
       [-1.55420519]]), array([[-0.43850763],
       [-0.29132015],
       [-1.54400591]]), array([[-0.14157566],
       [-0.02512072],
       [-1.60525797]]), array([[-0.02908071],
       [-0.01056125],
       [-1.58684873]]))


translation vectors:
(array([[-4.50019757],
       [ 1.29146576],
       [12.25454666]]), array([[-4.56000397],
       [-0.65985054],
       [15.4917626 ]]), a

In [10]:
f_x = mtx[0,0]
f_y = mtx[1,1]

print(f"Focal length in x direction, (f_x) = {f_x}")
print(f"Focal length in y direction, (f_y) = {f_y}")

Focal length in x direction, (f_x) = 4099.816464093187
Focal length in y direction, (f_y) = 4099.294774917002


In [9]:
box_size = 3
box_corners = np.array([
    [0, 0, 0],
    [box_size, 0, 0],
    [box_size, box_size, 0],
    [0, box_size, 0],
    [0, 0, -box_size],
    [box_size, 0, -box_size],
    [box_size, box_size, -box_size],
    [0, box_size, -box_size]
], dtype=np.float32)

for k, fname in enumerate(images):
    img = cv.imread(fname)

    h,  w = img.shape[:2]
    newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 0, (w,h))

    print(f"\n[{fname}] New Camera Matrix:")
    print(newcameramtx)

    # undistort
    dst = cv.undistort(img, mtx, dist, None, newcameramtx)
    cv.imwrite(f'undistorted_{fname}', dst)
    print(f"Saved undistorted_{fname}")


    # crop the image
    x, y, w, h = roi
    dst = dst[y:y+h, x:x+w]
    cv.imwrite(f'calibresult_{fname}.png', dst)

    # Draw the cube
    # Project the 3D box corners to 2D
    img_box_corners = [project_point_to_image(pt, rvecs[k], tvecs[k], newcameramtx) for pt in box_corners]

    # Draw lines between the box corners
    for i in range(4):
        cv.line(dst, img_box_corners[i], img_box_corners[(i + 1) % 4], (255, 0, 0), thickness=50)
        cv.line(dst, img_box_corners[i + 4], img_box_corners[((i + 1) % 4) + 4], (255, 0, 0), thickness=50)
        cv.line(dst, img_box_corners[i], img_box_corners[i + 4], (255, 0, 0), thickness=50)

    cv.waitKey(0)
    cv.imwrite(f'3d_{fname}', dst)
    print(f"Saved 3d_{fname}")

cv.destroyAllWindows()


[IMG_1052.JPG] New Camera Matrix:
[[4.66825618e+03 0.00000000e+00 2.16258441e+03]
 [0.00000000e+00 4.66417547e+03 2.86471718e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Saved undistorted_IMG_1052.JPG
Saved 3d_IMG_1052.JPG

[IMG_1053.JPG] New Camera Matrix:
[[4.66825618e+03 0.00000000e+00 2.16258441e+03]
 [0.00000000e+00 4.66417547e+03 2.86471718e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Saved undistorted_IMG_1053.JPG
Saved 3d_IMG_1053.JPG

[IMG_1056.JPG] New Camera Matrix:
[[4.56281102e+03 0.00000000e+00 2.11373660e+03]
 [0.00000000e+00 4.52722153e+03 2.78060063e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Saved undistorted_IMG_1056.JPG
Saved 3d_IMG_1056.JPG

[IMG_1049.JPG] New Camera Matrix:
[[4.66825618e+03 0.00000000e+00 2.16258441e+03]
 [0.00000000e+00 4.66417547e+03 2.86471718e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Saved undistorted_IMG_1049.JPG
Saved 3d_IMG_1049.JPG

[IMG_1047.JPG] New Camera Matrix:
[[4.66825618e+03 0.00000000e+00 2